## Swaps and Swaptions

Pre-requisites: Notebook 1.1, 1.2

In this notebook we will create and price **swaps**, **vanilla swaptions**, and **bermuda swaptions**. In that process we will

- use a contract with multiple **tracks**
- option to switch from one track to another
- use `HWFDModel` (a Finite Difference Hull-White Model)

API documentation:

- [Utilities to create swaps and swaptions](https://qablet.github.io/qablet-contracts/examples/rate/)
- [Hull White FD model](https://qablet-academy.github.io/intro/models/fd/#hull-white-model)

Let us start with the imports.

In [ ]:
import numpy as np
from qablet_contracts.timetable import timetable_from_dicts
from qablet_contracts.rate.swaption import simple_swap_period, bermuda_swaption_timetable
from qablet.hullwhite.fd import HWFDModel
from qablet.base.fixed import FixedModel

## Single Swap Period
Before we do that, let us start with a single swap period which pays a fixed rate, and receives a floating rate at the end of the period. e.g.

```
 time quantity
  0.5  - fixed_rate * 0.5
  0.5  + floating rate * 0.5
```

Instead of defining a unit that pays based on a floating rate convention (such as LIBOR or SOFR), we will substitute it with an economically equivalent representation, where we receive notional at the beginning of the period, and pay notional at the end of the period. The utility `simple_swap_period` creates such a pair of events. 

In [ ]:
events = simple_swap_period(ccy="USD", start=0, end=0.5, fixed_rate=.03)
timetable = timetable_from_dicts(events)
print(timetable["events"].to_pandas())

  track  time op  quantity unit
0         0.0  +     1.000  USD
1         0.5  +    -1.015  USD


We can price it using a fixed model, and at 2.977% (which is 3% semi-annual converted to continuous rate) we see that the price is zero.

In [ ]:
def flat_discount_crv(rate, tmax):
    times = np.array([0.0, tmax])
    rates = np.array([rate, rate])
    return ("ZERO_RATES", np.column_stack((times, rates)))

dataset = {
    "BASE": "USD",
    "ASSETS": {"USD": flat_discount_crv(.029777, 5)},
}

fixed_model = FixedModel()
price, _ = fixed_model.price(timetable, dataset)
print(f"price: {price:11.6f}")

price:   -0.000000


## Vanilla Swap
Now we can create a vanilla swap using this building block.

In [ ]:
def vanilla_swap_timetable(ccy, periods, fixed_rate, track="") -> dict:
    """create timetable for a vanilla swap."""
    events = []
    # payment events
    for start, end in zip(periods[0:-1], periods[1:]):
        events.extend(simple_swap_period(ccy, start, end, fixed_rate, track=track))
    return timetable_from_dicts(events)

periods = np.linspace(0.5, 2, 4)  # Start after 6 months, mature at 2 years, pay semi-annually (three periods).
fixed_rate = 0.03
swap_timetable = vanilla_swap_timetable("USD", periods, fixed_rate, "")

print(swap_timetable["events"].to_pandas())

  track  time op  quantity unit
0         0.5  +     1.000  USD
1         1.0  +    -1.015  USD
2         1.0  +     1.000  USD
3         1.5  +    -1.015  USD
4         1.5  +     1.000  USD
5         2.0  +    -1.015  USD


We can price this swap with different shocks to the interest rate curve.

In [ ]:
for shock in range(-2,3):
    dataset["ASSETS"]["USD"] = flat_discount_crv(.029777 + shock * .0001, 5)
    price, _ = fixed_model.price(swap_timetable, dataset)
    print(f"shock {shock:-2d}bps  price: {price:10.6f}")

shock -2bps  price:  -0.000292
shock -1bps  price:  -0.000146
shock  0bps  price:  -0.000000
shock  1bps  price:   0.000145
shock  2bps  price:   0.000291


## A Vanilla Swaption
A Vanilla Swaption is similar to the swap, with a choice in the beginning to enter the swap or not. This can be done using a **>** operation to end the contract with 0 payment, or continue.

In [ ]:
def swaption_timetable(ccy, periods, strike_rate, track="") -> dict:
    # expiration event at beginning
    events = [
        {
                "track": track,
                "time": periods[0],
                "op": ">",
                "quantity": 0,
                "unit": "USD"
            }
    ]
    # payment events
    for start, end in zip(periods[0:-1], periods[1:]):
        events.extend(simple_swap_period(ccy, start, end, strike_rate, track=track))
    return timetable_from_dicts(events)

swo_timetable = swaption_timetable("USD", periods, strike_rate=fixed_rate)
print(swo_timetable["events"].to_pandas())

  track  time op  quantity unit
0         0.5  >     0.000  USD
1         0.5  +     1.000  USD
2         1.0  +    -1.015  USD
3         1.0  +     1.000  USD
4         1.5  +    -1.015  USD
5         1.5  +     1.000  USD
6         2.0  +    -1.015  USD


We will price it with a Hull-White Finite-Difference model, which requires a mean reversion and volatility parameter in the dataset. For details of the model parameters see [Hull White FD model](https://qablet-academy.github.io/intro/models/fd/#hull-white-model).


In [ ]:
hwfd_dataset = {
    "BASE": "USD",
    "ASSETS": {"USD": flat_discount_crv(.029777, 5)},
    "FD": {
        "TIMESTEP": 1 / 250,
        "MAX_X": 0.20,
        "N_X": 75,
    },
    "HW": {
        "MEANREV": 0.1,
        "VOL": 0.03,
    },
}

hw_model = HWFDModel()
price, _ = hw_model.price(swo_timetable, hwfd_dataset)
print(f"price: {price:11.6f}")

price:    0.011118


Let's change the vols.

In [ ]:
for vol in np.linspace(0.0, 0.05, 6):
    hwfd_dataset["HW"]["VOL"] = vol
    price, _ = hw_model.price(swo_timetable, hwfd_dataset)
    print(f"vol: {vol:6.2f} price: {price:11.6f}")

vol:   0.00 price:    0.000000
vol:   0.01 price:    0.003645
vol:   0.02 price:    0.007394
vol:   0.03 price:    0.011118
vol:   0.04 price:    0.014835
vol:   0.05 price:    0.018547


## Bermuda Swaption
 In Vanilla swaption the holder gets the opportunity to enter into the swap at the beginning of the first period. However, in a Bermuda swaption the holder gets multiple opprtunities to enter a swap. In the variant shown here, known as a Co-terminal Bermuda swaption, the holder can exercise his option at the beginning of each swap period. Once exercised, the holder pays and receives all remaining payments of the swap.
 
The timetable below has two tracks, `.opt` and `.swp`. 
- All the payments happen on the `.swp` track. 
- The holder starts in the track `.opt`, and has an option in the beginning of each period, to switch to the `.swp` track. This is represented by having `.swp` as the target unit in each option event, instead of an asset or cash.

In [ ]:
periods = np.linspace(0.5, 2, 4)  # Start after 6 months, mature at 2 years, pay semi-annually (three periods).
fixed_rate = 0.03
bswo_timetable = bermuda_swaption_timetable("USD", periods, strike_rate=fixed_rate)

print(bswo_timetable["events"].to_pandas())

  track  time op  quantity  unit
0  .opt   0.5  >     1.000  .swp
1  .swp   0.5  +     1.000   USD
2  .swp   1.0  +    -1.015   USD
3  .opt   1.0  >     1.000  .swp
4  .swp   1.0  +     1.000   USD
5  .swp   1.5  +    -1.015   USD
6  .opt   1.5  >     1.000  .swp
7  .swp   1.5  +     1.000   USD
8  .swp   2.0  +    -1.015   USD


In [ ]:
vols = np.linspace(0.0, 0.1, 6)
vanilla_swo_prices = []
bermuda_swo_prices = []

for vol in vols:
    hwfd_dataset["HW"]["VOL"] = vol
    v_price, _ = hw_model.price(swo_timetable, hwfd_dataset)
    b_price, _ = hw_model.price(bswo_timetable, hwfd_dataset)
    vanilla_swo_prices.append(v_price)
    bermuda_swo_prices.append(b_price)
    print(f"vol: {vol:6.2f} vanilla: {v_price:11.6f} bermuda: {b_price:11.6f}")

vol:   0.00 vanilla:    0.000000 bermuda:    0.000000
vol:   0.02 vanilla:    0.007394 bermuda:    0.009367
vol:   0.04 vanilla:    0.014835 bermuda:    0.018766
vol:   0.06 vanilla:    0.022257 bermuda:    0.028196
vol:   0.08 vanilla:    0.029684 bermuda:    0.037677
vol:   0.10 vanilla:    0.037188 bermuda:    0.047243


## Suggested Exercises

- Create and price a callable bond, or a prepayable amortizing loan, or a cancellable swap.
- Show how the bermuda swaption is a hedge for the above products.